In [1]:
import pandas as pd
import numpy as np
import nbimporter
from datetime import datetime, timedelta
import glob
import os
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

# 1. Korrelation

## 1.1 Bestimmung korrelierender Feature

In [3]:
def correlation_matrix(endresult_df, categorical_columns, cut_off_limit):
    """
    Erstellt eine Korrelationsmatrix und filtert die Spalten basierend auf einem Korrelationsschwellenwert.

    Parameter:
    endresult_df (DataFrame): Der ursprüngliche DataFrame, der analysiert wird.
    categorical_columns (list): Eine Liste von kategorischen Spalten, die von der Korrelationsanalyse ausgeschlossen werden.
    cut_off_limit (float): Der Schwellenwert für die absolute Korrelation. Spalten mit einer Korrelation unterhalb dieses Werts
                           gegenüber der Zielspalte werden entfernt.

    Rückgabewert:
    DataFrame: Ein DataFrame, in dem die Spalten mit geringer Korrelation zur Zielspalte entfernt wurden.
    """

    # Entfernen von kategorischen Spalten aus der Korrelationsanalyse
    columns_to_keep = [col for col in endresult_df.columns if not any(excl in col for excl in categorical_columns)]

    # Erstellen eines DataFrames nur mit relevanten numerischen Spalten
    df_relevant = endresult_df[columns_to_keep]
    
    # Berechnen der Korrelationsmatrix
    correlation_matrix = df_relevant.corr()
    
    # Größe der Figur festlegen
    plt.figure(figsize=(10, 8))
    
    # Erstellen des Heatmaps für die Korrelationsmatrix
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
    
    # Titel hinzufügen
    plt.title('Korrelationsmatrix Heatmap')
    
    # Plot anzeigen
    plt.show()
    
    # Definieren der Zielspalte
    target_column = "Gesamt (Netzlast) [MWh] Originalauflösungen"
    
    # Spalten ermitteln, die eine geringe Korrelation zur Zielspalte haben
    columns_to_drop = correlation_matrix.index[abs(correlation_matrix[target_column]) < cut_off_limit].tolist()
    
    # Entfernen dieser Spalten aus dem relevanten DataFrame
    df_filtered = df_relevant.drop(columns=columns_to_drop)
    
    # Wenn du die Korrelationsmatrix für den gefilterten DataFrame neu plotten möchtest
    filtered_correlation_matrix = df_filtered.corr()
    
    plt.figure(figsize=(10, 8))
    
    # Erstellen des Heatmaps für die gefilterte Korrelationsmatrix
    sns.heatmap(filtered_correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
    
    # Titel hinzufügen
    plt.title('Gefilterte Korrelationsmatrix Heatmap')
    
    # Plot anzeigen
    plt.show()

    # Entfernen der nicht relevanten Spalten aus dem ursprünglichen DataFrame
    df_end = endresult_df.drop(columns=columns_to_drop)
    
    # Rückgabe des gefilterten DataFrames
    return df_end


## 1.2 Reduzierung der Dimensionen

In [2]:
def reducing_dimensionality(df, correlation_threshold, categorical_columns):
    """
    Reduziert die Dimensionalität eines DataFrames, indem hochkorrelierte Spalten basierend auf einem festgelegten 
    Korrelationsschwellenwert entfernt werden.

    Parameter:
    df (DataFrame): Der Eingabe-DataFrame, der analysiert und reduziert werden soll.
    correlation_threshold (float): Der Schwellenwert für die absolute Korrelation. Spaltenpaare mit einer höheren Korrelation
                                   werden identifiziert und eine der beiden Spalten wird entfernt.
    categorical_columns (list): Eine Liste von kategorischen Spalten, die von der Korrelationsanalyse ausgeschlossen werden.

    Rückgabewert:
    DataFrame: Ein DataFrame mit reduzierter Dimensionalität, bei dem hochkorrelierte Spalten entfernt wurden.
    """
    
    # Entfernen von kategorischen Spalten aus der Korrelationsanalyse
    columns_to_keep = [col for col in df.columns if not any(excl in col for excl in categorical_columns)]

    # Filtern des DataFrames, um nur numerische Spalten zu behalten
    df_filtered = df[columns_to_keep]
    
    # Berechnen der absoluten Korrelationsmatrix
    correlation_matrix = df_filtered.corr().abs()
    
    # Erstellen einer Maske für das obere Dreieck der Korrelationsmatrix
    upper_triangle_mask = np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)
    
    # Finden der Spaltenpaare, die eine Korrelation über dem Schwellenwert haben
    high_correlation_pairs = [(correlation_matrix.columns[i], correlation_matrix.columns[j])
                              for i, j in zip(*np.where(correlation_matrix > correlation_threshold))
                              if upper_triangle_mask[i, j]]
    
    # Identifizieren der Spalten, die entfernt werden sollen
    columns_to_drop = set([pair[1] for pair in high_correlation_pairs])
    
    # Entfernen der stark korrelierten Spalten aus dem DataFrame
    df_reduced = df_filtered.drop(columns=columns_to_drop)
    
    # Optional: Korrelationsmatrix für den reduzierten DataFrame plotten
    reduced_correlation_matrix = df_reduced.corr()
    
    # Größe der Figur festlegen
    plt.figure(figsize=(10, 8))
    
    # Erstellen des Heatmaps für die reduzierte Korrelationsmatrix
    sns.heatmap(reduced_correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
    
    # Titel hinzufügen
    plt.title('Heatmap der gefilterten Features')
    
    # Plot anzeigen
    plt.show()

    # Entfernen der korrelierten Spalten aus dem ursprünglichen DataFrame
    df_end = df.drop(columns=columns_to_drop)
    
    # Anzeigen des gefilterten DataFrames
    print(df_end)
    
    # Rückgabe des reduzierten DataFrames
    return df_end


# 2. Encodierung

## 2.1 Ordinal encoding

In [6]:
def ranking_countries_by_viewership(football_dic_copy):
    """
    Rangiert Länder basierend auf den Einschaltquoten in Deutschland und gibt ein Wörterbuch mit den Ländern und ihren Rangplätzen zurück.

    Parameter:
    football_dic_copy (dict): Ein Dictionary, das DataFrames für verschiedene Fußballspiele enthält. Jeder DataFrame sollte
                               Spalten für die Länder ("Land1", "Land2") und die Einschaltquote ("Einschaltquote (Deutschland)") enthalten.

    Rückgabewert:
    dict: Ein Wörterbuch, das die Länder als Schlüssel und ihre Rangplätze basierend auf den Einschaltquoten als Werte enthält.
    """
    
    football_match = []
    for key, match_set in football_dic_copy.items():
        # Entfernen von führenden und nachfolgenden Leerzeichen in den Ländernamen
        match_set["Land1"] = match_set["Land1"].str.strip()
        match_set["Land2"] = match_set["Land2"].str.strip()
        
        try:
            # Bereinigen und Konvertieren der Einschaltquote in eine numerische Form
            match_set["Einschaltquote (Deutschland)"] = match_set["Einschaltquote (Deutschland)"].str.strip(" Mio.")
            match_set["Einschaltquote (Deutschland)"] = match_set["Einschaltquote (Deutschland)"].str.replace(',', '.', regex=False)
            match_set["Einschaltquote (Deutschland)"] = pd.to_numeric(match_set["Einschaltquote (Deutschland)"], errors='coerce')
        
            # Hinzufügen des bereinigten DataFrames zur Liste
            football_match.append(match_set)
        except:
            # Fehler beim Verarbeiten der Einschaltquote ignorieren
            pass
    
    # Kombinieren aller DataFrames in der Liste in einen einzigen DataFrame
    result_df = pd.concat(football_match, ignore_index=True)
    
    # Auflösen der DataFrames, sodass jedes Land eine eigene Zeile erhält
    df_melted = pd.melt(result_df, id_vars=["Einschaltquote (Deutschland)"], value_vars=["Land1", "Land2"],
                        var_name='country_position', value_name='country')
    
    # Gruppieren nach Land und Summieren der Einschaltquoten
    country_views = df_melted.groupby('country')["Einschaltquote (Deutschland)"].sum().reset_index()
    
    # Rangieren der Länder nach den Gesamteinschaltquoten
    view_df = country_views.sort_values(by="Einschaltquote (Deutschland)", ascending=False).reset_index(drop=True)
    view_df["Rank"] = view_df.index + 1
    
    # Erstellen eines Wörterbuchs, das die Ländernamen den Rangplätzen zuordnet
    country_to_rank = dict(zip(view_df['country'], view_df['Rank']))

    return country_to_rank


## 2.2 Cyclic Encoding

In [4]:
def cyclic_encoding(df_not_2024, categorical_columns, columns_to_normalize):
    """
    Wendet zyklische Kodierung auf Zeitmerkmale an und normalisiert die angegebenen Spalten in einem DataFrame.

    Parameter:
    df_not_2024 (DataFrame): Der DataFrame, auf dem die zyklische Kodierung und Normalisierung angewendet werden.
    categorical_columns (list): Eine Liste von kategorischen Spalten, die möglicherweise nicht normalisiert werden.
    columns_to_normalize (list): Eine Liste von Spalten, die normalisiert werden sollen.

    Rückgabewert:
    DataFrame: Der DataFrame nach der zyklischen Kodierung und Normalisierung, wobei die Originalspalten der Zeitmerkmale entfernt wurden.
    """
    
    scaler = MinMaxScaler()
    
    # Normalisieren der angegebenen Spalten
    cyclic_normalized_df = df_not_2024.copy()
    cyclic_normalized_df[columns_to_normalize] = scaler.fit_transform(cyclic_normalized_df[columns_to_normalize])
    
    # Zyklische Kodierung für Zeitmerkmale
    cyclic_normalized_df['hour_sin'] = np.sin(2 * np.pi * cyclic_normalized_df['hour'] / 24)
    cyclic_normalized_df['hour_cos'] = np.cos(2 * np.pi * cyclic_normalized_df['hour'] / 24)
    
    cyclic_normalized_df['day_sin'] = np.sin(2 * np.pi * cyclic_normalized_df['day'] / 31)
    cyclic_normalized_df['day_cos'] = np.cos(2 * np.pi * cyclic_normalized_df['day'] / 31)
    
    cyclic_normalized_df['minute_sin'] = np.sin(2 * np.pi * cyclic_normalized_df['minute'] / 60)
    cyclic_normalized_df['minute_cos'] = np.cos(2 * np.pi * cyclic_normalized_df['minute'] / 60)
    
    cyclic_normalized_df['day_of_week_sin'] = np.sin(2 * np.pi * cyclic_normalized_df['day_of_week'] / 7)
    cyclic_normalized_df['day_of_week_cos'] = np.cos(2 * np.pi * cyclic_normalized_df['day_of_week'] / 7)

    # Entfernen der ursprünglichen Zeitmerkmal-Spalten, falls gewünscht
    cyclic_normalized_df.drop(['hour', 'day', 'minute', 'day_of_week'], axis=1, inplace=True)
    
    return cyclic_normalized_df
